In [1]:
## 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 필요 라이브러리 및 JDK 설치

1. JAVA jDK 설치
    
  Hadoop에서 JVM을 사용하기 때문에 SDK 필요
2. Spark.3.4.1
  
  pyspark 사용을 위한 Spark 다운
3. konlpy

  단어 추출을위한 형태소분석기
4. findspark
  
  pyspark가 정규 라이브러리로 인식이 안되기때문에 이를 해결하기 위한 라이브러리


In [2]:
# hadoop 이 JVM을 사용하기 때문에 sdk 필요
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
# spark down
!wget -q https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

In [4]:
# 압축해제
!tar xf spark-3.4.1-bin-hadoop3.tgz

In [5]:
#형태소 분석 라이브러리 knoply 다운
!pip3 install konlpy JPype1-py3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 41.6 MB/s eta 0:00:00
  Created wheel for JPype1-py3: filename=JPype1_py3-0.5.5.4-cp310-cp310-linux_x86_64.whl size=3258010 sha256=b5a92e50d3dbaa1a70957adb3f605a637a65b249ee41d73c83d7854068cbcd80
  Stored in directory: /root/.cache/pip/wheels/57/72/ea/b886a286a27c6e3c35ba9e9833b13abc5c5bdc0a9cad91e328
Successfully built JPype1-py3


In [6]:
# Install library for finding Spark
!pip install -q findspark

In [7]:
# 환경변수 설정
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.4.1-bin-hadoop3'

In [14]:
import findspark
# pyspark가 정규 library로 인식되지 않기 때문에 pyspark의 위치를 찾을 수 없음.
# finspark 라이브러리 import후 init()을 통해 pyspark의 위치를 찾도록 함
findspark.init()
findspark.find()

In [15]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

In [16]:
# Import the libary
from pyspark.sql.functions import col
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql import SparkSession
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt  # 형태소 분석기
import csv
import chardet
import math
import time
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re, os, string


### Spark Session 생셩


In [17]:
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

# Keyword 추출 과정

1. 데이터 가져오기 - 기사 데이터 / 불용어 사전


2. 기사 별 본문에서 명사 추출 + 불용어 제거


3. 명사를 하나의 String으로 통합

  ['쏙쏙', '비행기', '관찰', '도감', '항공기', '비행기', '뭐'] -> ['쏙쏙 비행기 관찰 항공기 비행기 뭐']

4. TF-IDF 계산

5. TOP 10 핵심 키워드 추출


In [18]:
# 기사 데이터
csv_file_path = '/content/drive/MyDrive/NewKids/article_test_data.csv'
# 불용어 사전
csv_stopword_path = '/content/drive/MyDrive/NewKids/불용어.csv'

# Constants
PUNCTUATION = """!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""
TOP_K_KEYWORDS = 10 # top k number of keywords to retrieve in a ranked document

In [19]:
# 1. 데이터 가져오기 - 기사 데이터
csv_file_path = '/content/drive/MyDrive/NewKids/article_test_data.csv'


def get_article(csv_file_path) :
  article_list = []

  # encoding
  rawdata = open(csv_file_path, 'rb').read()
  result = chardet.detect(rawdata)
  encoding = result['encoding']

  with open(csv_file_path, 'r', encoding=encoding) as csv_file :
      reader = csv.reader(csv_file)
      # 첫번째 행만 읽기
      for line in reader:
        article_list.append(line)

  return article_list

article_list = get_article(csv_file_path)

article_list[1]

['[‘쏙쏙’ 비행기 관찰 도감] 항공기, 비행기와 뭐가 다르지?',
 '',
 '\n        정리=현기성 기자 \n         ',
 '2023.09.05 23:00    ',
 'http://kid.chosun.com/site/data/img_dir/2023/09/05/2023090502610_thumb.jpg',
 '',
 '<div class="read-news article" id="article">\n                        \n                        <div class="Paragraph">\n                            항공기? 비행기? 어떤 게 옳은 말일까요? 둘 다 옳은 말이지만 쓰임은 다릅니다. 어떤 것은 항공기라 부르고 어떤 것은 비행기라 부르죠. 항공기와 비행기는 어떤 차이가 있을까요?\n                        </div>\n                        \n                        <div class="Paragraph">\n                                \t\t\n                                            <div class="center_img">\n                                                <dl style="width:770px;">\n                                                    <dd>\t\t\t\t\t\t\t  \n                                                        \n                                                        <img id="artImg1" src="http://kid.chosun.com/site/data/img_dir/2023/09/05/2023090

In [ ]:
# 1. 데이터 가져오기 - 불용어 사전
csv_file_path = '/content/drive/MyDrive/NewKids/불용어.csv'

def get_stopwords(csv_file_path):
  # 파일 인코딩을 감지하여 설정
  rawdata = open(csv_file_path, 'rb').read()
  result = chardet.detect(rawdata)
  encoding = result['encoding']

  # 불용어를 담을 list
  stop_word_list = []

  with open(csv_file_path, 'r', encoding=encoding) as csv_file :
      reader = csv.reader(csv_file)
      # 첫번째 행만 읽기
      first_row = next(reader)
      stop_word_list = first_row

  stop_word_set = set(stop_word_list)

  return stop_word_set

stop_word_set = get_stopwords(csv_file_path)

stop_word_set




In [ ]:
#2. 기사 별 본문에서 명사 추출

def get_keyword():
  start = time.time()
  keyword_list = []

  okt = Okt()
  # 0          1           2           3                  4              5            6            7
  #['title', 'sub_title', 'writer', 'published_date', 'thumbnail_img', 'content', 'html_content', 'imgs']

  for i in range(1, len(article_list)):
    # 본문에서 명사 추출
    res = okt.nouns(article_list[i][5])
    # 제목에서 명사 추출 후 본문 명사와 합치기
    res.extend(okt.nouns(article_list[i][0]))
    # rdd로 변환
    rdd = spark.sparkContext.parallelize(res)
    # 필터 적용(불용어 제거)
    rdd = rdd.filter(lambda word: word not in stop_word_set)

    keyword_list.append(rdd.collect())

  end = time.time()
  print(f"총 걸린 시간 : {end - start:.5f} sec")
  return keyword_list

keyword_list = get_keyword()
keyword_list[:10]

In [31]:
import numpy as np
np.array(keyword_list).ndim

<ipython-input-31-d1cf1d22ae97>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(keyword_list).ndim


1

In [22]:
def sort_coo(coo_matrix):
    """Sort a dict with highest score"""
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""

    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    # word index and corresponding tf-idf score
    for idx, score in sorted_items:

        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature, score
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]

    return results

In [23]:
def get_keywords(vectorizer, feature_names, doc):
    """Return top k keywords from a doc using TF-IDF method"""

    #generate tf-idf for the given document
    tf_idf_vector = vectorizer.transform([doc])


    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only TOP_K_KEYWORDS
    keywords=extract_topn_from_vector(feature_names,sorted_items,TOP_K_KEYWORDS)

    return list(keywords.keys())

In [24]:
#corpora = data['content'].to_list()
# 3. 명사를 하나의 String으로 통합
corpora = [' '.join(keyword) for keyword in keyword_list]
corpora[0]

'쏙쏙 비행기 관찰 도감 항공기 비행기 뭐'

In [25]:
# 4. TF-IDF 계산

start = time.time()
# Initializing TF-IDF Vectorizer with stopwords
vectorizer = TfidfVectorizer(smooth_idf=True, use_idf=True)

# Creating vocab with our corpora
# Exlcluding first 10 docs for testing purpose

#TF-IDF 벡터 행렬 -> 추천 알고리즘에 사용됨
matrix = vectorizer.fit_transform(corpora)

# Storing vocab
feature_names = vectorizer.get_feature_names_out()
end = time.time()

print(f"TF-IDF 계산 시간 : {end - start:.5f} sec")

TF-IDF 계산 시간 : 0.07384 sec


In [26]:
# 5. TOP10 핵심 키워드 추출
# RESULT
result = []
for doc in corpora[0:10]:
    df = {}
    df['id'] =
    df['content'] = doc
    df['top_keywords'] = get_keywords(vectorizer, feature_names, doc)
    result.append(df)

final = pd.DataFrame(result)
final

# 핵심 키워드가 10개가 안넘는것은 "본문"이 없는 포토기사 입니다.

,content,top_keywords
0,쏙쏙 비행기 관찰 도감 항공기 비행기 뭐,"[비행기, 항공기, 쏙쏙, 도감, 관찰]"
1,예상 특선 양서연 서울 서초 초 산문 김시연 충북 충주 남초 동시,"[특선, 충주, 양서연, 서초, 남초, 김시연, 산문, 충북, 동시, 예상]"
2,동시 모금,"[모금, 동시]"
3,연두 남매 엄마 수술 일주일 산골 체험 마을 살 누나 연두 남동생 단둘 생활 점 마...,"[연두, 마을, 산골, 남매, 사람, 물놀이, 일주일, 마음, 탈출, 정체]"
4,인류 바이러스 모든 변종 바이러스 만능 백신 연구 중,"[바이러스, 변종, 백신, 만능, 인류, 연구, 모든]"
5,무늬 친환경 꼼짝 마 유명 기업 싱,"[꼼짝, 무늬, 유명, 친환경, 기업]"
6,마음 거울 뜻 명심보감 조선시대 서당 교과서 마음 성현 가르침 때문 책 명심보감 내...,"[명심보감, 초등, 마음, 성현, 서당, 사자성어, 구절, 교과, 거울, 가르침]"
7,염색체 유전자 세포 속 중요 기관 사람 총 쌍 염색체 염색체 크기 형태 번 번 쌍 ...,"[염색체, 염기, 유전자, 서열, 게놈, 남성, 인간, 해독, 해석, 크기]"
8,어린이 안내서 담배 접 수 뇌 손상 학습 능력 기억 담당 뇌 세포 치명,"[안내서, 손상, 담배, 세포, 치명, 학습, 담당, 기억, 능력, 어린이]"
9,도전 후회 게 거 이준혁 서울대학교 체육관 체육 교육 학부 생 박일 승 씨 이준혁 ...,"[체육, 박씨, 물리학, 연구, 교수, 전공, 물리, 수학, 이씨, 지능]"
